In [ ]:
#!/usr/bin/env python

import os
import sys
import pickle as pk
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from kle import KLE
from gsa import sobol

# assume uniform prior used for the Freeze-Thaw model parameters in this study
# if using a different prior, please modify the log likelihood equations in the logpost function accordingly
from mcmc import AMCMC, logpost

myrc()

# set the UQTk installation path
os.environ['UQTK_INS']='/Users/username/Desktop/research/UQ/UQTk_new/UQTk-install'

# add UQTk to the Python path
sys.path.append('/Users/username/Desktop/research/UQ/UQTk_new/UQTk-install')

# For an ensemble of M model runs, and a data set of N observations,
# Load observational data
obs_data = np.loadtxt('obs_data.txt') # Size N x 1

# Load model ensemble data
model_ens = np.loadtxt('ydata.txt') # Size N x M

# Load parameter training range (min, max) and parameter ensemble
prange = np.loadtxt('prange.txt') # Size Ndim x 2, where Ndim is the number of parameters
pall = np.loadtxt('pall.txt')  # Size M x Ndim

# Get sizes
ngrid, nens = model_ens.shape
ngrid_, = obs_data.shape
nens_, ndim = pall.shape
ndim_, two = prange.shape



# Sanity check on sizes
assert(ngrid==ngrid_)
assert(ndim==ndim_)
assert(nens==nens_)
assert(two==2)

# Print useful info
print(f'Ensembles size : {nens}')
print(f'Parameter dim : {ndim}')
print(f'Grid size : {ngrid}')


# Diagnostic plot of model ensemble and obs. data
for iens in range(nens):
    plt.plot(range(ngrid), model_ens[:, iens], 'r-', lw=0.3, label='Model ensemble')
plt.plot(range(ngrid), obs_data, 'k-', lw=2, label='Obs. data')
plt.xlabel('')
plt.ylabel('Temperature')
handles,labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[-2:],labels[-2:])
plt.savefig('model_ens.png')
plt.clf()

######################################################################
# Construct KL approximation
######################################################################

# Build KL expansion
kl = KLE()
kl.build(model_ens)

# Pick the first neig eigenvalues that explain 95% of variance
explained_variance = np.cumsum(kl.eigval)/np.sum(kl.eigval)
neig = np.where(explained_variance>0.95)[0][0]+1
# neig = X # you can also set neig to a specific number for testing purposes
print(f'Number of eigenvalues requested : {neig}')

# Plot eigenvalues and KL modes
kl.plot_eig()
kl.plot_klmodes(imodes=range(neig))

# Check the KL approximation
model_kl = kl.eval(neig=neig)

# Diagnostic plot to make sure model_ens and model_kl are close
isams = [33, 66] # for example sample number 33 and 66
for isam in isams:
    plt.plot(range(ngrid), model_ens[:, isam], 'b-', lw=1, label='Model data')
    plt.plot(range(ngrid), model_kl[:, isam], 'g-', lw=1, label='KL apprx.')
    plt.xlabel('')
    plt.ylabel('Temperature')
    plt.legend()
    plt.savefig('model_kl_'+str(isam)+'.png')
    plt.clf()


In [ ]:
######################################################################
# Construct PC surrogates for KL modes
######################################################################

# Scale the input parameters to [-1,1] for easier surrogate building and MCMC
qall = 2.0 * scaleDomTo01(pall, prange) - 1.0
# Outputs to build surrogates for are the \xi variables of KL expansion
yall = kl.xi[:, :neig]
nout = yall.shape[1] # number of outputs to build surrogates for is neig


# Build surrogate for y as a function of q
# Set the order of the polynomial chaos expansion
order = 4
uqpcdir = os.environ['UQTK_INS']+'/examples/uqpc/'
np.savetxt('ytrain.dat', yall)
np.savetxt('qtrain.dat', qall)
np.savetxt('ptrain.dat', pall)

# Build the surrogate using UQPC by calling uq_pc.py
command = uqpcdir+'uq_pc.py -r offline_post -p prange.txt -i m -m bcs -s rand -n ' + str(nens) +' -v 0 -t ' + str(order) + ' -e 1.e-7'
os.system(command)

# Loading surrogate results
results = pk.load(open('results.pk', 'rb'))


# Check the surrogate accuracy plotting the eigenmodes
yall_pred = multi_surrogate(qall, results)
for iout in range(nout):
    plt.plot(yall[:,iout], yall_pred[:, iout], 'o', label=f'Output (eigenmode) # {iout+1}')
plt.plot([-4,4], [-4,4], 'k--', lw=2)
plt.xlabel('True xi')
plt.ylabel('Surrogate xi')
plt.legend()
plt.savefig('model_surr.png')
plt.clf()

# Check the whole KLPC surrogate accuracy plotting in the physical space
model_klpc = kl.eval(xi=yall_pred, neig=neig)
isams = [33, 66] # for example sample number 33 and 66
for isam in isams:
    plt.plot(range(ngrid), model_ens[:, isam], 'b-', lw=1, label='Model data')
    plt.plot(range(ngrid), model_kl[:, isam], 'g-', lw=1, label='KL apprx.')
    plt.plot(range(ngrid), model_klpc[:, isam], 'r-', lw=1, label='KLPC apprx.')
    plt.xlabel('Days, t')
    plt.ylabel('Model QoI')
    plt.legend()
    plt.savefig('model_klpc_'+str(isam)+'.png')
    plt.clf()



In [ ]:
######################################################################
# MCMC calibration using KLPC surrogate
######################################################################

# Project obs. data to KL modes to get obs. data eigenmodes \nu
nu = kl.project(obs_data[:, np.newaxis])[:, :neig] # nu is 1 x Neig, but in principle one can accommodate more than one observational sequence

# Set the initial parameters and run MCMC
nmcmc = 100000  # number of MCMC samples requested
gamma = 0.01  # gamma parameter (jump size) of aMCMC
t0 = 1000  # when adaptation, i.e. proposal covariance update, starts
tadapt = 100  # how often adaptive proposal covariance is updated

# # give different weight based on eigenvalues
# # instructed by Khachik, 04/24/2023
f = 1
recp_eig = np.reciprocal(kl.eigval)
sigma = []
for i in range(neig):
    sigma.append(f*np.sqrt(neig)*np.sqrt(recp_eig[i]/np.sum(recp_eig[:neig])))


# Initial parameters and covariance for MCMC
param_ini = np.random.rand(ndim)  # initial parameter values
cov_ini = np.diag(0.01*param_ini**2) # initial covariance


# Set dictionary info for posterior computation
lpinfo = {'model': multi_surrogate,
          'otherpars': results, 'yd': nu.T,
          'ltype': 'classical',
          'lparams': {'sigma': sigma}}


# Set MCMC
my_amcmc = AMCMC()
my_amcmc.setParams(param_ini=param_ini, cov_ini=cov_ini,
                   t0=t0, tadapt=tadapt,
                   gamma=gamma, nmcmc=nmcmc)


# Run MCMC inference
mcmc_results = my_amcmc.run(logpost, lpinfo)
#mcmc_results = {'chain' : samples, 'mapparams' : cmode, 'maxpost' : pmode, 'accrate' : acc_rate}

# Postprocess MCMC
np.savetxt('chain.txt', mcmc_results['chain'])
np.savetxt('logpm.txt', mcmc_results['logpostm'])

# # Get MAP sample
xi_map = multi_surrogate(mcmc_results['mapparams'], results).reshape(1,-1)
model_map = kl.eval(xi=xi_map, neig=neig)

# Get MCMC samples, usually we discard initial parts and 'thin' the chain, e.g. pick every 10th sample
xi_mcmc = multi_surrogate(mcmc_results['chain'][nmcmc//2::10], results)
model_mcmc = kl.eval(xi=xi_mcmc, neig=neig)

model_mean = np.average(model_mcmc, axis=1)
model_std = np.std(model_mcmc, axis=1)

plt.plot(range(ngrid), model_map, 'b-', label='MAP')
p, = plt.plot(range(ngrid), model_mean, 'g-', label='Mean')
lc = lighten_color(p.get_color(), 0.5)
plt.fill_between(range(ngrid), model_mean - model_std,
                 model_mean + model_std, color=lc, zorder=-1000, alpha=1.0, label='StDev')
plt.plot(range(ngrid), obs_data, 'k-', lw=2, label='Obs. data')

plt.xlabel('')
plt.ylabel('Temperature')
plt.legend()
plt.savefig('fit.png')
plt.clf()

# Quick and dirty visualization of parameter PDFs
# Save scaled versions of parameter samples for plotting
np.savetxt('mapparams_phys.txt', scale01ToDom(0.5 * (1. + mcmc_results['mapparams'].reshape(1,-1)), prange).T)
np.savetxt('chain_phys.txt', scale01ToDom(0.5 * (1. + mcmc_results['chain']), prange))
os.system(os.environ['UQTK_INS']+'/examples/iuq/plot_pdfs.py -p chain_phys.txt -g prange.txt -t tri -b 100 -e 10 -l mapparams_phys.txt ')
# see pdf_tri.eps

# Quick and dirty visualization of prior and posterior predictions
# Same prior and posterior predictions for plotting
np.savetxt('sam_prior.txt', model_ens.T)
np.savetxt('sam_post.txt', model_mcmc.T)
os.system(os.environ['UQTK_INS']+'/examples/iuq/plot_shade.py -y sam_prior.txt -z sam_post.txt -d obs_data.txt')
# see fit_shade.eps

# Cleanup (a bit risky, but worth it, these are irrelevant after the dust is settled)
os.system('rm -rf cfs coeff.dat data jointsens.dat mainsens.dat totsens.dat lambdas.dat mi.dat mindex.dat mindex_new.dat pccf.dat PCcoeff.dat ptrain.dat qtrain.dat regparams.dat selected.dat sp_mindex* target varfrac.dat xcheck.dat xdata.dat ycheck.dat ydata.dat ytrain.dat')


In [ ]:
######################################################################
# save KLPC results
######################################################################
np.savetxt('kl_mean.txt',kl.mean)
np.savetxt('kl_modes.txt',kl.kl_modes)
np.savetxt('kl_weights2.txt',kl.weights2.reshape(-1,1))
np.savetxt('kl_eigval.txt',kl.eigval)
np.savetxt('model_map.txt',model_map)
np.savetxt('model_mean.txt',model_mean)
np.savetxt('ytrain_surr.txt', yall_pred)
np.savetxt('ytrain.txt', yall)
np.savetxt('nu.txt',nu)
np.savetxt('xi_map.txt',xi_map)
np.savetxt('sigma.txt', sigma)

In [ ]:
######################################################################
# Global Sensitivity Analysis (PC based)
######################################################################

# For BCS, all outputs may have different multiindices, so this joins them in a union set...
# ... so that one can switch sums in the KLPC expansion
pccf_all = results['pcmi'][0]
mindex_all = results['pcmi'][1]
mindex_common, cfs_common = micf_join(mindex_all, pccf_all)
assert(np.sum(mindex_common[0, :]) == 0) # sanity check that first multiindex is 0th term

# Sensitivities for eigenmodes
allsens_eig_pc = multisens(mindex_common, cfs_common, sens_type='total')
np.savetxt('allsens_eig_pc.txt', allsens_eig_pc)
os.system(os.environ['UQTK_INS']+'/examples/iuq/plot_sens.py -s allsens_eig_pc.txt ; mv sens.eps sens_eig_pc.eps')
# see sens_eig_pc.eps

# Sensitivities for the actual model
# This is implementation of Eq (6) in KLPC document
cfs_glob = kl.eval(xi=cfs_common.T, neig=neig) - kl.mean.reshape(-1,1)
cfs_glob[:, 0] = kl.mean + cfs_glob[:, 0]

allsens_pc = multisens(mindex_common, cfs_glob, sens_type='total')
np.savetxt('allsens_pc.txt', allsens_pc)
os.system(os.environ['UQTK_INS']+'/examples/iuq/plot_sens.py -s allsens_pc.txt; mv sens.eps sens_pc.eps')
# see sens_pc.eps

######################################################################
# Global Sensitivity Analysis (Sobol based - this will be useful if non-PC surrogates are used)
######################################################################

npar = 1000 # Number of samples requested
domain = np.tile(np.array([-1.,1.]), (ndim, 1))
SensMethod = sobol(domain)
xsam = SensMethod.sample(npar)
ysam_eig = multi_surrogate(xsam, results)

# Sensitivities for eigenmodes
allsens_eig_sobol = np.empty((neig, ndim))
# mainsens_eig_sobol = np.empty((neig, ndim))
for i in range(neig):
    sens = SensMethod.compute(ysam_eig[:, i])
    allsens_eig_sobol[i, :] = sens['total'] # or 'main'
    # mainsens_eig_sobol[i, :] = sens['main']


np.savetxt('allsens_eig_sobol.txt', allsens_eig_sobol)
os.system(os.environ['UQTK_INS']+'/examples/iuq/plot_sens.py -s allsens_eig_sobol.txt; mv sens.eps sens_eig_sobol.eps')
# see sens_eig_sobol.eps

# Sensitivities for the actual model
ysam_model = kl.eval(xi=ysam_eig, neig=neig).T
allsens_sobol = np.empty((ngrid, ndim))
for i in range(ngrid):
    sens = SensMethod.compute(ysam_model[:, i])
    allsens_sobol[i, :] = sens['total'] # or 'main'

np.savetxt('allsens_sobol.txt', allsens_sobol)
os.system(os.environ['UQTK_INS']+'/examples/iuq/plot_sens.py -s allsens_sobol.txt; mv sens.eps sens_sobol.eps')
# see sens_sobol.eps

In [ ]:
mainsens_eig_sobol = np.empty((neig, ndim))
for i in range(neig):
    sens = SensMethod.compute(ysam_eig[:, i])
    # allsens_eig_sobol[i, :] = sens['total'] # or 'main'
    mainsens_eig_sobol[i, :] = sens['main']


In [ ]:
jointsens_eig_sobol = np.empty((neig, ndim, ndim))
for i in range(neig):
    sens = SensMethod.compute(ysam_eig[:, i])
    jointsens_eig_sobol[i, :, :] = sens['jointt']


In [ ]:
######################################################################
# Visualize Global Sensitivity Results (PC based)
######################################################################

# Main parameter sensitivities
sensitivity = np.sum(mainsens_eig_sobol, axis=0)
# Joint parameter sensitivities
jointsensitivity = np.sum(jointsens_eig_sobol, axis=0)

# Circle properties
center = (0, 0)  # Center coordinates of the circle
radius = 1  # Radius of the circle

# Calculate dot positions on the circumference
theta = np.linspace(0, 2 * np.pi, len(sensitivity) + 1)[:-1]
x = center[0] + radius * np.cos(theta)
y = center[1] + radius * np.sin(theta)

# Create figure and axis
fig, ax = plt.subplots()

# Plot main circle
circle = plt.Circle(center, radius, color='black', linewidth=1, fill=False)
ax.add_artist(circle)

# Plot dots
dot_color = 'red'
dot_edgecolor = 'dimgray'
dot_sizes = np.array(sensitivity) * 0.1  # Scale the sensitivity values to determine dot sizes
for i, (dot_x, dot_y) in enumerate(zip(x, y)):
    transparency = 0
    dot = plt.Circle((dot_x, dot_y), dot_sizes[i] / 2, edgecolor=dot_edgecolor, facecolor=dot_color)
    ax.add_artist(dot)

# Plot lines
line_widths = np.array(jointsensitivity) * 10  # Scale the sensitivity values to determine line widths
for i in range(len(sensitivity) - 1):
    for j in range(i + 1, len(sensitivity)):
        transparency = 0.5
        ax.plot([x[i], x[j]], [y[i], y[j]], color='red', linewidth=line_widths[i,j], alpha=transparency)

# Set axis limits and aspect ratio
ax.set_aspect('equal')
ax.set_xlim([-1.5, 1.5])
ax.set_ylim([-1.5, 1.5])

# Add labels to dots
# Replace with your parameter labels
dot_labels = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']
rr = 0.15
for i, label in enumerate(dot_labels):
    ax.text(x[i]+rr*np.cos(theta[i]), y[i]+rr*np.sin(theta[i]), label, horizontalalignment='center', verticalalignment='center')

# Add title and grid
ax.set_title('Parameter Sensitivity')
ax.grid(False)
ax.axis('off')


plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate a random N by N matrix
N = 21  # Replace with your matrix size
matrix = jointsensitivity

# Create the figure and axis
fig, ax = plt.subplots()

# Plot the matrix with colors
inverse_cmap = plt.colormaps.get_cmap('hot').reversed()
im = ax.imshow(matrix, cmap=inverse_cmap)  # You can choose any colormap you prefer

# Add colorbar
cbar = fig.colorbar(im)

# Customize axis tick labels
xtick_labels = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']  # Replace with your desired tick labels
ytick_labels = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']  # Replace with your desired tick labels
ax.set_xticks(np.arange(N))
ax.set_yticks(np.arange(N))
ax.set_xticklabels(xtick_labels)
ax.set_yticklabels(ytick_labels)

# Set title
ax.set_title('Joint Sensitivity Visualization')
ax.tick_params(axis='both', labelsize=12)

# Show the plot
plt.show()
